In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import MinMaxScaler
from tpot.builtins import StackingEstimator
from xgboost import XGBClassifier

/Users/ryanashcraft/opt/anaconda3/lib/python3.7/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [19]:
tpot_data = pd.read_csv('mach_learn_df.csv')

tpot_data = tpot_data.drop(columns=['Unnamed: 0','City'])

tpot_data = tpot_data.rename(columns={'Offense_Type': 'target'})

tpot_data.head()

,Hour,target,Premise,Tract,Month,Day_of_Week,Temperature,Weather
0,0,Burglary/Robbery,Residence or House,313100,1,Monday,46.56,Clouds
1,0,Burglary/Robbery,Apartment,321300,1,Monday,46.56,Clouds
2,0,Burglary/Robbery,"Road, Street, or Sidewalk",432801,1,Monday,46.56,Clouds
3,0,Assault,"Road, Street, or Sidewalk",330700,1,Monday,46.56,Clouds
4,0,Theft,"Church, Synagogue, or Temple Parking Lot",312800,1,Monday,46.56,Clouds


In [14]:
# # Assign X (data) and y (target)
# X = tpot_data.drop(["target"], axis=1)
# y = tpot_data["target"]
# print(X.shape, y.shape)

(309959, 7) (309959,)


0         Burglary/Robbery
1         Burglary/Robbery
2         Burglary/Robbery
3                  Assault
4                    Theft
                ...       
309954               Theft
309955        Intimidation
309956               Theft
309957        Intimidation
309958               Theft
Name: target, Length: 309959, dtype: object

In [17]:
from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
# tpot_data = label_encoder.fit(tpot_data)
# # transformed_y = label_encodery.transform(y)

# tpot_data

In [26]:
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,tpot_data,y=None):
        return self # not relevant here

    def transform(self,tpot_data):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = tpot_data.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,tpot_data,y=None):
        return self.fit(tpot_data,y).transform(tpot_data)
    
tpot_data = MultiColumnLabelEncoder(columns = ['Day_of_Week', 'Premise', 'Tract', 'Weather', 'Hour', 'Month', 'target', 'Temperature']).fit_transform(tpot_data)

tpot_data

,Hour,target,Premise,Tract,Month,Day_of_Week,Temperature,Weather
0,0,2,125,156,0,1,654,1
1,0,2,9,180,0,1,654,1
2,0,2,131,385,0,1,654,1
3,0,1,131,204,0,1,654,1
4,0,7,33,153,0,1,654,1
...,...,...,...,...,...,...,...,...
309954,23,7,126,491,4,3,2182,6
309955,23,3,126,486,4,3,2182,6
309956,23,7,18,486,4,3,2182,6
309957,23,3,126,361,4,3,2182,6


In [27]:
# NOTE: Make sure that the outcome column is labeled 'target' in the data file
# tpot_data = pd.read_csv('../transfusion.csv', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'], random_state=None)

# Average CV score on the training set was: 0.5198433049211877
exported_pipeline = make_pipeline(
    MinMaxScaler(),
    StackingEstimator(estimator=RandomForestClassifier(bootstrap=False, criterion="entropy", max_features=0.7000000000000001, min_samples_leaf=20, min_samples_split=6, n_estimators=100)),
    XGBClassifier(learning_rate=0.1, max_depth=8, min_child_weight=20, n_estimators=100, nthread=1, subsample=0.8500000000000001)
)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)


In [37]:
exported_pipeline.score(testing_features, testing_target)

0.43187508065556846

In [46]:
prediction_labels = label_encodery.inverse_transform(results[:50])
actual_labels = label_encodery.inverse_transform(testing_target[:50])

In [47]:
actual_vs_predicts = zip(actual_labels, prediction_labels)

for actual, predict in actual_vs_predicts:
    print(f'Actual: {actual} - Prediction: {predict}')
    if actual == predict:
        print('Correct!')
    else:
        print('Boo :(')

Actual: Theft - Prediction: Theft
Correct!
Actual: Intimidation - Prediction: Vandalism
Boo :(
Actual: Theft - Prediction: Theft
Correct!
Actual: Theft - Prediction: Theft
Correct!
Actual: Vandalism - Prediction: Burglary/Robbery
Boo :(
Actual: Intimidation - Prediction: Burglary/Robbery
Boo :(
Actual: Burglary/Robbery - Prediction: Burglary/Robbery
Correct!
Actual: Theft - Prediction: Theft
Correct!
Actual: Intimidation - Prediction: Vandalism
Boo :(
Actual: Theft - Prediction: Theft
Correct!
Actual: Theft - Prediction: Theft
Correct!
Actual: Theft - Prediction: Theft
Correct!
Actual: Burglary/Robbery - Prediction: Assault
Boo :(
Actual: Assault - Prediction: Burglary/Robbery
Boo :(
Actual: Theft - Prediction: Theft
Correct!
Actual: Assault - Prediction: Theft
Boo :(
Actual: Theft - Prediction: Theft
Correct!
Actual: Assault - Prediction: Vandalism
Boo :(
Actual: Burglary/Robbery - Prediction: Burglary/Robbery
Correct!
Actual: Assault - Prediction: Vandalism
Boo :(
Actual: Intimidatio